In [1]:
import os, sys, codecs, time, datetime
import fileinput
import os.path
from io import StringIO
from subprocess import call


from docopt import docopt
from propsde.applications.viz_tree import DepTreeVisualizer

import propsde.applications.run as run
import sys
if sys.version_info[0] >= 3:
    unicode = str

In [2]:
import pickle

In [3]:
with open('polizeiberichte.pkl', 'rb') as f:
    data = pickle.load(f, encoding="utf8")

In [4]:
text_example = data[0]["text1"]

In [5]:
text_example = text_example.replace(". ", ".\n")

In [6]:
print(text_example)

Unbekannte griffen in der vergangenen Nacht einen Kiosk in Kreuzberg an.
Gegen 0.40 Uhr bemerkten Zeugen eine Gruppe von ungefähr sieben Männern, die mehrere Steine gegen das Schaufenster des Geschäftes an der Graefestraße warfen und die Scheibe beschädigten.
Anschließend schütteten die Angreifer eine Flüssigkeit gegen das Fenster und entzündeten sie.
Die anschließenden Flammen sollen bis in das erste Obergeschoss gereicht haben, wurden jedoch von Anwohnern gelöscht.
Die Tatverdächtigen flüchteten unerkannt in Richtung Hasenheide.
Die Ermittlungen führt ein Brandkommissariat beim Landeskriminalamt Berlin.


In [7]:
file = codecs.open("examples.txt", encoding='utf8')
gs = run.parseSentences(file)

running mate-tools
lemmatizing
parsing + tagging
collapsing
java -jar ext/org.jobimtext.collapsing.jar -i tmp/parsed.conll06 -o tmp -sf -l de -r ext/resources/german_modified.txt -f c -np -nt
SVP
MO
UFF
UFF
UFF
MO


In [8]:
for g,tree in gs: 
    for prop in g.getPropositions('pdf'):
        curProp = '{0}:({1})'.format(prop.pred,
                                      ", ".join([rel + ":" + arg for rel,arg in sorted(prop.args,key=lambda rel:prop.rel_order(rel[0]))]))
        print(curProp)

angreifen:(subj:Unbekannte , dobj:einen Kiosk in Kreuzberg , prep_in:der vergangenen Nacht )
bemerkt:(subj:Zeugen , dobj:eine Gruppe von ungefähr sieben Männern , die mehrere Steine gegen das Schaufenster des Geschäftes an der Graefestraße warfen und die Scheibe beschädigten , prep_gegen:0:40 Uhr )
werfen:(subj:ungefähr sieben Männern , und die Scheibe beschädigten , dobj:mehrere Steine , prep_gegen:das Schaufenster des Geschäftes an der Graefestraße )
entzünden:(subj:die Angreifer , dobj:sie )
Anschließend schütten:(subj:die Angreifer , dobj:eine Flüssigkeit gegen das Fenster )
und:(conj_und:Anschließend schütteten die Angreifer eine Flüssigkeit gegen das Fenster , conj_und:die Angreifer entzündeten sie )
reichen:(subj:Die anschließenden Flammen , prep_in:das erste Obergeschoss )
unerkannt flüchten:(subj:Die Tatverdächtigen , prep_in:Richtung Hasenheide )
führen:(subj:ein Brandkommissariat , dobj:Die Ermittlungen , prep_bei:Landeskriminalamt Berlin )


In [9]:
prop = g.getPropositions('pdf')[0]
print(prop.args)
print(prop.pred)

[['dobj', 'Die Ermittlungen '], ['subj', 'ein Brandkommissariat '], ['prep_bei', 'Landeskriminalamt Berlin ']]
führen


In [33]:
test = gs[0][0]
print(test)

Unbekannte griffen in der vergangenen Nacht einen Kiosk in Kreuzberg an .
0	1,Unbekannte	NN	0	0	subj,87
3	5,vergangenen	ADJA	0	0	mod,4
4	6,Nacht	NN	0	0	prep_in,87
6	8,Kiosk	NN	0	0	dobj,87
8	10,Kreuzberg	NE	0	0	prep_in,6
87	2,griffen;11,an		1	1	



In [32]:
for t in test:
    print(t)

<TABLE BORDER="0" CELLSPACING="0"><TR><TD>Unbekannte<FONT POINT-SIZE="7">[1]</FONT></TD></TR><TR><TD><FONT POINT-SIZE="10">det: </FONT></TD></TR><TR><TD><FONT POINT-SIZE="10">indefinite</FONT></TD></TR></TABLE>
<TABLE BORDER="0" CELLSPACING="0"><TR><TD>vergangenen<FONT POINT-SIZE="7">[5]</FONT></TD></TR><TR><TD><FONT POINT-SIZE="10">det: </FONT></TD></TR><TR><TD><FONT POINT-SIZE="10">indefinite</FONT></TD></TR></TABLE>
<TABLE BORDER="0" CELLSPACING="0"><TR><TD>Nacht<FONT POINT-SIZE="7">[6]</FONT></TD></TR><TR><TD><FONT POINT-SIZE="10">det: </FONT></TD></TR><TR><TD><FONT POINT-SIZE="10">indefinite</FONT></TD></TR></TABLE>
<TABLE BORDER="0" CELLSPACING="0"><TR><TD>Kiosk<FONT POINT-SIZE="7">[8]</FONT></TD></TR><TR><TD><FONT POINT-SIZE="10">det: </FONT></TD></TR><TR><TD><FONT POINT-SIZE="10">indefinite</FONT></TD></TR></TABLE>
<TABLE BORDER="0" CELLSPACING="0"><TR><TD>Kreuzberg<FONT POINT-SIZE="7">[10]</FONT></TD></TR><TR><TD><FONT POINT-SIZE="10">det: </FONT></TD></TR><TR><TD><FONT POINT-